## Fleet Assignment + Revenue Management


### The weighted sum method is used for multiobjective optimization.

\begin{equation}
\text{Maximize} \;\; 

\alpha \cdot Revenue - \beta \cdot Cost, \;\; \text{where} \;\; \alpha + \beta = 1

\tag{1}
\end{equation}

### Note:
\- We normalize the weight to make it easier to compare the ratio between them.

\- In addition, this model does not consider network flowing or repositioning of the aircrafts, which means that we assume there are sufficient fleet at each origin where needed.

\- Our model works on parameters stated below, and fixed schedule and fleet information are required.

----------------------

### Decision Variables

* $x_{i,j}$: binary indicator whether flight leg $i$ is assigned to the aircraft type $j$

* $s^E_i$: economy seats sold on flight leg $i$
* $s^B_i$: business seats sold on flight leg $i$

### Sets and Indices
* $F$ : set of flight legs (e.g., IST-YVR, YVR-IST, etc.), indexed by $i$.
* $G$ : set of aircraft types (e.g., B777-300ER, B787-9), indexed by $j$.

### Parameters (Inputs)

* $c_{i,j}$: operating cost of assigning aircraft $j$ to flight leg $i$
* $N_j$: number of available aircraft of fleet type $j$

* $p^E_i$: ticket price for economy seat classes of flight leg $i$ in CAD dollars
* $p^B_i$: ticket price for business seat classes of flight leg $i$ in CAD dollars

* $c^E_j$: economy seat capacity of aircraft type $j$
* $c^B_j$: business seat capacity of aircraft type $j$

* $d^E_i$: economy seat demand on flight $i$
* $d^B_i$: business seat demand on flight $i$

* A list of flight schedule that contains flight leg and demand
* $R_j$: the maximum flight range of fleet type $j$

-----------

### Basic Objective Function

\begin{equation}
\text{Maximize}\;\; 

\alpha \left( \sum_{i \in F}{p^E_i \cdot s^E_i + p^B_i \cdot s^B_i} \right)
 - \beta \sum_{i \in F}\sum_{j \in G}{c_{i,j} \cdot x_{i,j}}

\tag{2}
\end{equation}

#### Operating Cost Breakdown

\begin{equation}
c_{i,j} = (f_j \cdot p_f + w_j + m_j) \cdot h_i
\tag{3}
\end{equation}

* $f_j$: fuel burn rate of fleet type $j$ in kg/hour
* $p_f$: fuel price per kg
* $w_j$: crew cost of fleet type $j$
* $m_j$: maintenance cost of fleet type $j$ per flight

* $h_i$: block time of flight leg $i$ in hours, $h_i = d_i / v_j$
* $d_i$: distance of flight leg $i$ in miles
* $v_j$: block speed of fleet type $j$

### Constraints

Subject to

\begin{align}

\sum_{j \in G}{x_{i,j} = 1} \text{, } \;\; \forall i \in F    \tag{4} \\

x_{i,j} = 0 \quad \text{if } d_i > R_j     \tag{5} \\

s^E_i \leq \sum_{j \in G}{c^E_j \cdot x_{i,j}} \text{, } \;\; \forall i \in F    \tag{6} \\

s^B_i \leq \sum_{j \in G}{c^B_j \cdot x_{i,j}} \text{, } \;\; \forall i \in F    \tag{7} \\

s^E_i \leq d^E_i \text{, } \;\; \forall i \in F    \tag{8} \\

s^B_i \leq d^B_i \text{, } \;\; \forall i \in F    \tag{9} \\

x_{i,j} \in {0,1} \text{, } \;\; s^E_i, s^B_i \geq 0    \tag{10}

\end{align}

(4) Flight coverage: each flight leg must be assigned to exactly one fleet type.

(5) Aircraft range feasibility: a fleet type cannot be assigned to a flight leg whose distance is greater than the maximum flight range of fleet type $j$.

(6) Economy class capacity

(7) Business class capacity

(8) Economy class demand

(9) Business class demand

(10) Decision variable bounds

-------------

In [1]:
import pandas as pd

In [2]:
flights = pd.DataFrame([
    {"fid":"TK75",  "orig":"IST", "dest":"YVR", "dist_mi":5973.9, "demE":100, "demB":80, "ecoP": 866, "bizP": 2700},
    {"fid":"TK76",  "orig":"YVR", "dest":"IST", "dist_mi":5973.9, "demE":124, "demB":54, "ecoP": 897, "bizP": 2780},

    {"fid":"TK001", "orig":"IST", "dest":"JFK", "dist_mi":5000.0, "demE":211, "demB":37, "ecoP": 680, "bizP": 2100}, 
    {"fid":"TK002", "orig":"JFK", "dest":"IST", "dist_mi":5000.0, "demE":197, "demB":91, "ecoP": 675, "bizP": 2210},

    {"fid":"TK193", "orig":"IST", "dest":"LHR", "dist_mi":1550.0, "demE":229, "demB":55, "ecoP": 320, "bizP": 1790},
    {"fid":"TK194", "orig":"LHR", "dest":"IST", "dist_mi":1550.0, "demE":186, "demB":17, "ecoP": 317, "bizP": 1765},
])


fleet = pd.DataFrame([
    {
        "type":"A350-900",
        "eco_seats":330,
        "bis_seats":110, 
        "fuel_kg_hr":7650,
        "maint_usd_hr":2200,
        "pilot_usd_hr":1800,
        "cabin_usd_hr":1200,
        "N":3, 
        "range_mi":9551
    },
    {
        "type":"B777-300ER", 
        "eco_seats":288,
        "bis_seats":104, 
        "fuel_kg_hr":7500,
        "maint_usd_hr":2200,
        "pilot_usd_hr":1800,
        "cabin_usd_hr":1200,
        "N":2, 
        "range_mi":8705
    },
    {
        "type":"B787-9", 
        "eco_seats":256,
        "bis_seats":40,
        "fuel_kg_hr":5600,
        "maint_usd_hr":2000,
        "pilot_usd_hr":1700,
        "cabin_usd_hr":1100,
        "N":2, 
        "range_mi":7565
    },
])

print(flights)
print(fleet)

     fid orig dest  dist_mi  demE  demB  ecoP  bizP
0   TK75  IST  YVR   5973.9   100    80   866  2700
1   TK76  YVR  IST   5973.9   124    54   897  2780
2  TK001  IST  JFK   5000.0   211    37   680  2100
3  TK002  JFK  IST   5000.0   197    91   675  2210
4  TK193  IST  LHR   1550.0   229    55   320  1790
5  TK194  LHR  IST   1550.0   186    17   317  1765
         type  eco_seats  bis_seats  fuel_kg_hr  maint_usd_hr  pilot_usd_hr  \
0    A350-900        330        110        7650          2200          1800   
1  B777-300ER        288        104        7500          2200          1800   
2      B787-9        256         40        5600          2000          1700   

   cabin_usd_hr  N  range_mi  
0          1200  3      9551  
1          1200  2      8705  
2          1100  2      7565  
